<a href="https://colab.research.google.com/github/ShrutiParate/GEN_AI_Digitide/blob/main/Week1/Day3_Basics_of_GenAI/HuggingFace_textgen_gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 110.5 MB/s eta 0:00:00


In [ ]:
import torch
print("Torch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print("GPU:", torch.cuda.get_device_name(0))
    except:
        pass

Torch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Choose a model. gpt2 is small and will run fine in Colab.
# If you want faster/more fluent results, try "gpt2-medium" or "gpt2-large" (they require more memory).
model_name = "gpt2"

print("Loading model:", model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
# GPT-2 family doesn't have a pad token by default — use eos token as pad
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)
model.eval()
print(f"Model loaded on {device}")


Loading model: gpt2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Model loaded on cuda


In [ ]:
import time
from typing import List, Tuple

def generate_text(prompt: str,
                  max_new_tokens: int = 150,
                  temperature: float = 0.7,
                  top_k: int = 50,
                  top_p: float = 0.95,
                  num_return_sequences: int = 1) -> Tuple[List[str], float]:
    """Return (list_of_outputs, elapsed_seconds)."""
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items()}
    t0 = time.time()
    out_ids = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=temperature,
        top_k=top_k,
        top_p=top_p,
        pad_token_id=tokenizer.eos_token_id,
        num_return_sequences=num_return_sequences,
        no_repeat_ngram_size=2
    )
    elapsed = time.time() - t0
    outputs = [tokenizer.decode(ids, skip_special_tokens=True) for ids in out_ids]
    return outputs, elapsed


In [ ]:
import textwrap

prompts = [
    "tell me superscience",
    "what is the meaning of life"

]

for p in prompts:
    outs, elapsed = generate_text(p, max_new_tokens=150)
    print("PROMPT:", p)
    for idx, o in enumerate(outs, 1):
        print(f"\n--- Output {idx} ---")
        wrapped = textwrap.fill(o.strip(), width=80)
        print(wrapped)
    print(f"\n[generation time: {elapsed:.2f} seconds]\n")


PROMPT: tell me superscience

--- Output 1 ---
tell me superscience. My first thought was that the idea of a superman in a
suit, a superhero, who could walk around like a real person and walk like the
Terminator, was something that I'd never thought about before. But I quickly
realized that wasn't what the story was about. That it was a story about two
superhumans who have just been transformed into different people. They're a lot
like one another and we're in the same world, but we have to fight to save the
world from this new world and to have a good time. It's a great movie.  How did
you approach the series? -I'm a big fan of the comic book series. I've read a
few other comics about superheroes, and I like

[generation time: 1.54 seconds]

PROMPT: what is the meaning of life

--- Output 1 ---
what is the meaning of life in the world?  "The basic question is that, if life
is, in most cases, a question of value to the individual, and if there is a
value in all things, then this value 